In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime
import pytz
from bs4 import BeautifulSoup
import spacy
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter
from IPython.display import display

# Load SpaCy model & Sentiment Analyzer
nlp = spacy.load("en_core_web_sm")
sia = SentimentIntensityAnalyzer()

# Get today's date in IST
current_date_ist = datetime.now(pytz.timezone('Asia/Kolkata')).date()

# Function to extract article content
def fetch_article_content(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        content = ' '.join(p.text.strip() for p in paragraphs if p.text.strip())
        return content if content else "N/A"
    except Exception:
        return "Error fetching content"

# Function to extract Named Entities and Sentiment
def process_text(text):
    doc = nlp(text)
    sentiment_score = sia.polarity_scores(text)["compound"]
    sentiment = "Positive" if sentiment_score > 0.2 else "Negative" if sentiment_score < -0.2 else "Neutral"

    entities = {"PERSON": [], "LOC": [], "ORG": [], "MISC": []}
    for ent in doc.ents:
        if ent.label_ in ["PERSON"]:
            entities["PERSON"].append(ent.text)
        elif ent.label_ in ["GPE", "LOC"]:
            entities["LOC"].append(ent.text)
        elif ent.label_ in ["ORG"]:
            entities["ORG"].append(ent.text)
        else:
            entities["MISC"].append(ent.text)

    # Extract Important Words
    words = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop]
    keyword_counts = Counter(words)
    keywords = [word for word, freq in keyword_counts.most_common(10)]  # Top 10 Keywords

    return sentiment, sentiment_score, entities, keywords

# Function to fetch articles metadata and content
def Fetch_Articles_Metadata(channel_names, channels_xml_links):
    data = []
    for i, links in enumerate(channels_xml_links):
        for xml_url in links:
            try:
                response = requests.get(xml_url, headers={'User-Agent': 'Mozilla/5.0'})
                response.encoding = 'utf-8'
                root = ET.fromstring(response.text)
                namespace = {
                    'sitemap': 'http://www.sitemaps.org/schemas/sitemap/0.9',
                    'news': 'http://www.google.com/schemas/sitemap-news/0.9'
                }
                for url in root.findall('sitemap:url', namespace):
                    title = url.find('news:news/news:title', namespace)
                    title = title.text.strip() if title is not None else "N/A"
                    link = url.find('sitemap:loc', namespace)
                    link = link.text.strip() if link is not None else "N/A"
                    published_datetime = url.find('news:news/news:publication_date', namespace)
                    published_datetime = published_datetime.text.strip() if published_datetime is not None else None
                    if published_datetime:
                        try:
                            parsed_datetime = datetime.strptime(published_datetime, "%Y-%m-%dT%H:%M:%S%z")
                            parsed_datetime_ist = parsed_datetime.astimezone(pytz.timezone('Asia/Kolkata'))

                            # **Check if the article was published today**
                            if parsed_datetime_ist.date() == current_date_ist:
                                content = fetch_article_content(link)
                                sentiment, sentiment_score, entities, keywords = process_text(content)
                                data.append({
                                    'Source': channel_names[i],
                                    'Title': title,
                                    'Link': link,
                                    'Published': parsed_datetime_ist.strftime('%Y-%m-%d %H:%M:%S'),
                                    'Sentiment': sentiment,
                                    'Sentiment Score': sentiment_score,
                                    'Persons': ', '.join(set(entities["PERSON"])),
                                    'Locations': ', '.join(set(entities["LOC"])),
                                    'Organizations': ', '.join(set(entities["ORG"])),
                                    'Miscellaneous': ', '.join(set(entities["MISC"])),
                                    'Keywords': ', '.join(set(keywords))
                                })
                        except ValueError:
                            continue
            except Exception:
                continue
    return pd.DataFrame(data)

# Define News Sources
channel_names = ["The Hindu", "Hindustan Times", "NDTV", "News18", "Zee News", "Jagran", "Firstpost", "Indian Express", "LiveMint", "business standard","India.com", "Indiatoday", "bhaskar", "dnaindia"]
channels_xml_links = [
    ["https://www.thehindu.com/sitemap/googlenews/all/all.xml"],
    ["https://www.hindustantimes.com/sitemap/news.xml"],
    ["https://www.ndtv.com/sitemap.xml"],
    ["https://www.news18.com/commonfeeds/v1/eng/sitemap/google-news/today.xml"],
    ["https://zeenews.india.com/sitemap.xml"],
    ["https://english.jagran.com/news-sitemap.xml"],
    ["https://www.firstpost.com/commonfeeds/v1/mfp/sitemap/google-news.xml"],
    ["https://indianexpress.com/news-sitemap.xml"],
    ["https://www.livemint.com/sitemap/today.xml"],
    ['https://www.business-standard.com/sitemap/news-sitemap.xml'],
    ['https://www.india.com/google-news-sitemap.xml'],
    ['https://www.indiatoday.in/sitemapindex.xml'],
    ['https://www.bhaskarenglish.in/sitemaps-v1--sitemap-google-news-1.xml'],
    ['https://www.dnaindia.com/googlenews.xml']
]

# Fetch Data
df = Fetch_Articles_Metadata(channel_names, channels_xml_links)
display(df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import plotly.express as px
import geopandas as gpd
from wordcloud import WordCloud
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

if not df.empty:
    ### 1. SENTIMENT ANALYSIS BAR CHART ###
    plt.figure(figsize=(8, 5))
    sns.countplot(x='Sentiment', data=df, palette={"Positive": "green", "Neutral": "gray", "Negative": "red"})
    plt.title("1. Sentiment Analysis of News Articles")
    plt.xlabel("Sentiment")
    plt.ylabel("Number of Articles")
    plt.show()

    ### 2. TOP KEYWORDS WORDCLOUD ###
    wordcloud_text = " ".join(df['Persons'].dropna())
    wordcloud = WordCloud(width=800, height=400, background_color='black', colormap='coolwarm').generate(wordcloud_text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title("2. Top Keywords in News Articles")
    plt.show()

    ### 3. TOP LOCATIONS MENTIONED IN NEWS ###
    locations = df['Locations'].dropna().str.split(', ').explode()
    top_locations = locations.value_counts().head(10)

    plt.figure(figsize=(10, 5))
    sns.barplot(x=top_locations.index, y=top_locations.values, palette="magma")
    plt.xticks(rotation=45)
    plt.title("3. Top 10 Locations in News Articles")
    plt.xlabel("Location")
    plt.ylabel("Number of Mentions")
    plt.show()

    ### 4. ORGANIZATIONS IN NEWS ###
    organizations = df['Organizations'].dropna().str.split(', ').explode()
    top_organizations = organizations.value_counts().head(10)

    plt.figure(figsize=(10, 5))
    sns.barplot(x=top_organizations.index, y=top_organizations.values, palette="coolwarm")
    plt.xticks(rotation=45)
    plt.title("4. Top 10 Organizations in News Articles")
    plt.xlabel("Organization")
    plt.ylabel("Number of Mentions")
    plt.show()

    ### 5. Source vs Sentiment ###
    plt.subplots(figsize=(8, 8))
    plt.title("5. Source vs Sentiment")
    df_2dhist = pd.DataFrame({
        x_label: grp['Sentiment'].value_counts()
        for x_label, grp in df.groupby('Source')
    })
    sns.heatmap(df_2dhist, cmap='viridis')
    plt.xlabel('Source')
    _ = plt.ylabel('Sentiment')

    ### 6. Source vs Sentiment Score ###
    figsize = (12, 1.2 * len(df['Source'].unique()))
    plt.figure(figsize=figsize)
    plt.title("6. Source vs Sentiment Score")
    sns.violinplot(df, x='Sentiment Score', y='Source', inner='stick', palette='Dark2')
    sns.despine(top=True, right=True, bottom=True, left=True)

    ### 7. Sentiment vs Sentiment Score ###
    figsize = (12, 1.2 * len(df['Sentiment'].unique()))
    plt.figure(figsize=figsize)
    plt.title("7. Sentiment vs Sentiment Score")
    sns.violinplot(df, x='Sentiment Score', y='Sentiment', inner='stick', palette='Dark2')
    sns.despine(top=True, right=True, bottom=True, left=True)


    df_resampled = df['Sentiment Score'].resample('30T').mean()
    plt.figure(figsize=(16, 8))
    plt.plot(df_resampled.index, df_resampled, linestyle='-', marker='o', alpha=0.7)

    plt.title("7. Sentiment Score Over Time (Smoothed - 30 Min Avg)", fontsize=14)
    plt.xlabel("Time of Day", fontsize=12)
    plt.ylabel("Sentiment Score", fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.gca().spines[['top', 'right']].set_visible(False)

    plt.show()



    df['Published'] = pd.to_datetime(df['Published'])
    df.set_index('Published', inplace=True)

    # Resampling Sentiment Scores over 30-minute intervals
    df_resampled = df['Sentiment Score'].resample('30T').mean()

    plt.figure(figsize=(16, 8))
    plt.plot(df_resampled.index, df_resampled, linestyle='-', marker='o', alpha=0.7, color='blue')

    plt.title("8. Sentiment Score Trend Over Time (30-Min Avg)", fontsize=14)
    plt.xlabel("Time", fontsize=12)
    plt.ylabel("Sentiment Score", fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.axhline(0, color='gray', linestyle='--')  # Neutral line
    plt.gca().spines[['top', 'right']].set_visible(False)

    plt.show()





else:
    print("No data available for visualization.")
